In [1]:
!pip install simpletransformers
import os
import matplotlib.pyplot as plt
import copy
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,DataCollatorWithPadding
from datasets import load_dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam,Nadam
from datasets import Dataset, DatasetDict

from transformers import DataCollatorWithPadding

from sklearn.model_selection import train_test_split
from collections import Counter
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape,MultiHeadAttention,Layer,Activation, Multiply, Lambda, Permute


from tensorflow.keras import Input, Model, regularizers
!pip install transformers==4.36.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.6 

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 27.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
# !pip install nlpaug
# import nlpaug.augmenter.word as naw

In [2]:
train = pd.read_csv("/content/rumour1.csv")
print(train.shape)
train.sample(5)

import pandas as pd
from sklearn.model_selection import train_test_split

model_checkpoint = "roberta-base"  # Use RoBERTa model
batch_size = 8

(2570, 26)


In [3]:

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [4]:
def find_sentiment_positive(tweet):
   return sia.polarity_scores(tweet)["pos"]

In [5]:
vader_positive_sentiments_source_text = train.source_text.apply(find_sentiment_positive)

In [6]:
def find_sentiment_negative(tweet):
   return sia.polarity_scores(tweet)["neg"]
vader_negative_sentiments_source_text = train.source_text.apply(find_sentiment_negative)

In [7]:
def find_sentiment_neutral(tweet):
   return sia.polarity_scores(tweet)["neu"]
vader_neutral_sentiments_source_text = train.source_text.apply(find_sentiment_neutral)

In [8]:
vader_positive_sentiments_reply_text = train.reply_text.apply(find_sentiment_positive)
vader_negative_sentiments_reply_text = train.reply_text.apply(find_sentiment_negative)
vader_neutral_sentiments_reply_text = train.reply_text.apply(find_sentiment_neutral)

In [9]:
train['v_p_s'] = 2*vader_positive_sentiments_source_text

train['v_neg_s'] = 2*vader_negative_sentiments_source_text

train['v_neu_s'] = 2*vader_neutral_sentiments_source_text
train.head()
train['v_p_r'] = 2*vader_positive_sentiments_reply_text

train['v_neg_r'] = 2*vader_negative_sentiments_reply_text

train['v_neu_r'] = 2*vader_neutral_sentiments_reply_text
train.head()

train = train[['source_text', 'reply_text', 'label', 'v_p_s',	'v_neg_s',	'v_neu_s'	,'v_p_r',	'v_neg_r',	'v_neu_r']]

In [10]:
train.head()

,source_text,reply_text,label,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r
0,France: 10 people dead after shooting at HQ of...,@euronews @TradeDesk_Steve A French crime of p...,2,0.000,0.424,1.576,0.388,0.452,1.162
1,#Ferguson chief let most of day go by b4 discl...,@leonlohr @LisaBloom Yep!! Splitting image in ...,2,0.230,0.312,1.458,0.332,0.000,1.668
2,#ISIS Media account posts picture claiming to ...,@ArmedResearch Which accounts?,2,0.000,0.666,1.334,0.000,0.000,2.000
3,Uber Sydney trips from CBD will be free for ri...,@Uber_Sydney so free or surge pricing? #confused,2,0.462,0.000,1.538,0.750,0.000,1.250
4,BREAKING Charlie Hebdo latest: 11 dead 10 woun...,"I'm in London right now, feel free to make an ...",2,0.000,0.446,1.554,0.404,0.000,1.596


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
npy_file_path = '/content/drive/My Drive/Clip_embeddings/clip_embeddings_source_text_rumoureval.npy' # Replace with the actual file path
loaded_source_data = np.load(npy_file_path)
loaded_source_data.shape

(2570, 512)

In [13]:
npy_file_path = '/content/drive/My Drive/Clip_embeddings/clip_embeddings_reply_text_rumoureval.npy'  # Replace with the actual file path
loaded_reply_data = np.load(npy_file_path)
loaded_reply_data.shape

(2570, 512)

In [14]:
# Load or generate your 2D NumPy array
df = pd.DataFrame(columns=['clip_source_text'])

# Iterate through the rows of the 2D array and add each row (with each element incremented by 1) to the DataFrame
for row in loaded_source_data:
    incremented_row = [element + 1 for element in row]
    df = df.append({'clip_source_text': incremented_row}, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-14-cafc9295fada>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_source_text': incremented_row}, ignore_index=True)
<ipython-input-14-cafc9295fada>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_source_text': incremented_row}, ignore_index=True)
<ipython-input-14-cafc9295fada>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_source_text': incremented_row}, ignore_index=True)
<ipython-input-14-cafc9295fada>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_source_text': incremented_ro

In [15]:
type(df['clip_source_text'].iloc[0])

list

In [16]:
clip_source_text_train= df['clip_source_text']

In [17]:
# Load or generate your 2D NumPy array
df = pd.DataFrame(columns=['clip_reply_text'])

# Iterate through the rows of the 2D array and add each row (with each element incremented by 1) to the DataFrame
for row in loaded_reply_data:
    incremented_row = [element + 1 for element in row]
    df = df.append({'clip_reply_text': incremented_row}, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-17-46427582f16b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_reply_text': incremented_row}, ignore_index=True)
<ipython-input-17-46427582f16b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_reply_text': incremented_row}, ignore_index=True)
<ipython-input-17-46427582f16b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_reply_text': incremented_row}, ignore_index=True)
<ipython-input-17-46427582f16b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'clip_reply_text': incremented_row}, 

In [18]:
clip_reply_text_train= df['clip_reply_text']

In [19]:
# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(train ,test_size=0.1, random_state=42)

print(test_data.shape)
test_data.tail(5)
train_data, val_data  = train_test_split(train_data,test_size=0.1, random_state=1, shuffle=False)
train_data.to_csv('train_clean.csv',index=False)
val_data.to_csv('val_clean.csv',index=False)
test_data.to_csv('test_clean.csv',index=False)

dataset = load_dataset('csv', data_files={'train': './train_clean.csv','valid':'./val_clean.csv','test':'./test_clean.csv'})

dataset

(257, 9)


Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r'],
        num_rows: 2081
    })
    valid: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r'],
        num_rows: 232
    })
    test: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r'],
        num_rows: 257
    })
})

In [20]:
dataset = load_dataset('csv', data_files={'train': './train_clean.csv','valid':'./val_clean.csv','test': './test_clean.csv'})
type(dataset)

datasets.dataset_dict.DatasetDict

In [25]:
import torch

# Define the Euclidean distance function
def euclidean_distance(embedding1, embedding2):
    return torch.norm(torch.tensor(embedding1) - torch.tensor(embedding2), dim=-1)

# Compute Euclidean distance
euclidean_distances = euclidean_distance(clip_source_text_train, clip_reply_text_train)


In [26]:
euclidean_distances

tensor([ 8.0322,  8.4515, 11.7889,  ...,  8.7823,  9.4128,  8.3981],
       dtype=torch.float64)

In [27]:
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function_source(records):
    tokenized = tokenizer(records['source_text'], truncation=True,padding='max_length', return_token_type_ids=True, max_length=128)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'token_type_ids': tokenized['token_type_ids']
    }


pre_tokenizer_columns_source = set(dataset["train"].features)

pre_tokenizer_columns_source

encoded_dataset_source = dataset.map(preprocess_function_source, batched=True, )

print(encoded_dataset_source)

tokenizer_columns_source = list(set(encoded_dataset_source["train"].features) - pre_tokenizer_columns_source)

tokenizer_columns_source.append('v_p_s')
tokenizer_columns_source.append('v_neg_s')
tokenizer_columns_source.append('v_neu_s')




print("Columns added by tokenizer:", tokenizer_columns_source)

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 2081
    })
    valid: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 232
    })
    test: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 257
    })
})
Columns added by tokenizer: ['attention_mask', 'input_ids', 'token_type_ids', 'v_p_s', 'v_neg_s', 'v_neu_s']


In [28]:
def preprocess_function_reply(records):
    tokenized = tokenizer(records['reply_text'], truncation=True,padding='max_length', return_token_type_ids=True, max_length=128)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'token_type_ids': tokenized['token_type_ids']
    }


pre_tokenizer_columns_reply = set(dataset["train"].features)

pre_tokenizer_columns_reply

encoded_dataset_reply = dataset.map(preprocess_function_reply, batched=True, )

print(encoded_dataset_reply)

tokenizer_columns_reply = list(set(encoded_dataset_reply["train"].features) - pre_tokenizer_columns_reply)


tokenizer_columns_reply.append('v_p_r')
tokenizer_columns_reply.append('v_neg_r')
tokenizer_columns_reply.append('v_neu_r')


print("Columns added by tokenizer:", tokenizer_columns_reply)

DatasetDict({
    train: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 2081
    })
    valid: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 232
    })
    test: Dataset({
        features: ['source_text', 'reply_text', 'label', 'v_p_s', 'v_neg_s', 'v_neu_s', 'v_p_r', 'v_neg_r', 'v_neu_r', 'input_ids', 'attention_mask', 'token_type_ids'],
        num_rows: 257
    })
})
Columns added by tokenizer: ['attention_mask', 'input_ids', 'token_type_ids', 'v_p_r', 'v_neg_r', 'v_neu_r']


In [29]:
!pip install nlpaug

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
from collections import Counter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 3.0 MB/s eta 0:00:00


In [30]:
import pandas as pd

# Assuming df is your DataFrame
class_counts = train_data['label'].value_counts()

print("Number of occurrences of each class:")
print(class_counts)



batch_size = 32
max_length = 128

Number of occurrences of each class:
2    576
0    535
3    512
1    458
Name: label, dtype: int64


In [31]:
#source
input_ids_train_source = np.array(encoded_dataset_source['train']['input_ids'])
token_type_ids_train_source = np.array(encoded_dataset_source['train']['token_type_ids'])
attention_mask_train_source = np.array(encoded_dataset_source['train']['attention_mask'])
v_p_s_train = np.array(encoded_dataset_source['train']['v_p_s'])
v_neg_s_train = np.array(encoded_dataset_source['train']['v_neg_s'])
v_neu_s_train = np.array(encoded_dataset_source['train']['v_neu_s'])

#reply
input_ids_train_reply = np.array(encoded_dataset_reply['train']['input_ids'])
token_type_ids_train_reply = np.array(encoded_dataset_reply['train']['token_type_ids'])
attention_mask_train_reply = np.array(encoded_dataset_reply['train']['attention_mask'])
v_p_r_train = np.array(encoded_dataset_reply['train']['v_p_r'])
v_neg_r_train = np.array(encoded_dataset_reply['train']['v_neg_r'])
v_neu_r_train = np.array(encoded_dataset_reply['train']['v_neu_r'])

In [32]:
#source
input_ids_valid_source = np.array(encoded_dataset_source['valid']['input_ids'])
token_type_ids_valid_source = np.array(encoded_dataset_source['valid']['token_type_ids'])
attention_mask_valid_source = np.array(encoded_dataset_source['valid']['attention_mask'])
v_p_s_v = np.array(encoded_dataset_source['valid']['v_p_s'])
v_neg_s_v = np.array(encoded_dataset_source['valid']['v_neg_s'])
v_neu_s_v = np.array(encoded_dataset_source['valid']['v_neu_s'])

#reply
input_ids_valid_reply = np.array(encoded_dataset_reply['valid']['input_ids'])
token_type_ids_valid_reply = np.array(encoded_dataset_reply['valid']['token_type_ids'])
attention_mask_valid_reply = np.array(encoded_dataset_reply['valid']['attention_mask'])
v_p_r_v = np.array(encoded_dataset_reply['valid']['v_p_r'])
v_neg_r_v = np.array(encoded_dataset_reply['valid']['v_neg_r'])
v_neu_r_v = np.array(encoded_dataset_reply['valid']['v_neu_r'])

In [33]:
#source
input_ids_test_source = np.array(encoded_dataset_source['test']['input_ids'])
token_type_ids_test_source = np.array(encoded_dataset_source['test']['token_type_ids'])
attention_mask_test_source = np.array(encoded_dataset_source['test']['attention_mask'])
v_p_s_test = np.array(encoded_dataset_source['test']['v_p_s'])
v_neg_s_test = np.array(encoded_dataset_source['test']['v_neg_s'])
v_neu_s_test = np.array(encoded_dataset_source['test']['v_neu_s'])

#reply
input_ids_test_reply = np.array(encoded_dataset_reply['test']['input_ids'])
token_type_ids_test_reply = np.array(encoded_dataset_reply['test']['token_type_ids'])
attention_mask_test_reply = np.array(encoded_dataset_reply['test']['attention_mask'])
v_p_r_test = np.array(encoded_dataset_reply['test']['v_p_r'])
v_neg_r_test = np.array(encoded_dataset_reply['test']['v_neg_r'])
v_neu_r_test = np.array(encoded_dataset_reply['test']['v_neu_r'])

In [37]:
def euclidean_distance(embedding1, embedding2):
    return torch.norm(torch.tensor(embedding1).float() - torch.tensor(embedding2).float(), dim=-1)

In [84]:
# Compute Euclidean distance
euclidean_distances_train = euclidean_distance(input_ids_train_source, input_ids_train_reply)
euclidean_distances_valid = euclidean_distance(input_ids_valid_source, input_ids_valid_reply)
euclidean_distances_test = euclidean_distance(input_ids_test_source, input_ids_test_reply)

In [39]:
euclidean_distances_train

tensor([ 79572.6875,  83136.0625,  83980.1328,  ...,  98930.3203,
         54739.5859, 114553.3281])

In [40]:
model_checkpoint2 = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint2)
labels=  ['support', 'query', 'deny','comment']
tokenized_labels=tokenizer(labels,
          return_tensors='tf',
          return_token_type_ids=True,truncation=True,padding='max_length',  max_length=3)
tokenized_labels

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  101,  1619,   102],
       [  101, 15027,   102],
       [  101,  9762,   102],
       [  101,  7368,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=int32)>}

In [41]:
max_length = 128
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Activation, Multiply, Lambda, Permute
from tensorflow.keras import regularizers

# Define the BERT model
#model_checkpoint = "bert-base-cased"
model_checkpoint2 = "bert-base-cased"

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint2)


# Get the BERT base model
bert_model2 = model.bert

from tensorflow.keras.layers import Layer

class BERTWrapperLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BERTWrapperLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs, **kwargs):
        input_ids, token_type_ids, attention_mask = inputs
        bert_output = self.bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        return bert_output.last_hidden_state
bert_last_hidden_state_labels = BERTWrapperLayer(bert_model2)([tokenized_labels['input_ids'],tokenized_labels['token_type_ids'], tokenized_labels["attention_mask"]])

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
arr1 = bert_last_hidden_state_labels
arr2 = np.mean(arr1, axis=1)

In [43]:
arr2.shape

(4, 768)

In [44]:
# Pass your input through the BERT model wrapper
input_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source")
token_type_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids_source")
attention_mask_source = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source")
v_p_s = Input(shape=(1,), dtype=tf.float32, name="v_p_s")
v_neg_s = Input(shape=(1,), dtype=tf.float32, name="v_neg_s")
v_neu_s = Input(shape=(1,), dtype=tf.float32, name="v_neu_s")
input_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply")
token_type_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="token_type_ids_reply")
attention_mask_reply = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply")
v_p_r = Input(shape=(1,), dtype=tf.float32, name="v_p_r")
v_neg_r = Input(shape=(1,), dtype=tf.float32, name="v_neg_r")
v_neu_r = Input(shape=(1,), dtype=tf.float32, name="v_neu_r")
arr2_input = Input(shape=(4,768), dtype=tf.float32, name="arr2_input")
euclidean_distances_input = Input(shape=(1,), dtype=tf.float32, name="euclidean_distances_input")
# Pass your input through the BERT model wrapper

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)
roberta_model=model.roberta

#deberta_pooler_output, deberta_last_hidden_state = DeBERTaWrapperLayer(deberta_model)([input_ids, attention_mask])
roberta_last_hidden_state_source = BERTWrapperLayer(roberta_model)([input_ids_source,token_type_ids_source, attention_mask_source])
roberta_last_hidden_state_reply = BERTWrapperLayer(roberta_model)([input_ids_reply,token_type_ids_reply, attention_mask_reply])

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
# layer = MultiHeadAttention(num_heads=2, key_dim=2)
# cross_attention_reply = layer(roberta_last_hidden_state_reply, roberta_last_hidden_state_reply)
# print(cross_attention_reply.shape)

In [ ]:
# layer = MultiHeadAttention(num_heads=2, key_dim=2)
# cross_attention_source = layer(roberta_last_hidden_state_reply, roberta_last_hidden_state_source)
# print(cross_attention_source.shape)

In [86]:
import tensorflow as tf

class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQ = tf.keras.layers.Dense(embed_size)
        self.WK = tf.keras.layers.Dense(embed_size)
        self.WV = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WO = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value = inputs

        batch_size = tf.shape(query)[0]

        # Linear projections
        query = self.WQ(query)
        key = self.WK(key)
        value = self.WV(value)

        # Split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention, attention_weights = self.scaled_dot_product_attention(query, key, value)

        # Merge heads
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embed_size))

        # Linear projection
        output = self.WO(concat_attention)

        return output, attention_weights

# Example usage
seq_len = 10
batch_size = 32
embed_size = 64
num_heads = 8

cross_attention_layer = CrossAttentionLayer(embed_size, num_heads)

# Dummy input tensors
input_query = roberta_last_hidden_state_reply
input_key = roberta_last_hidden_state_source
input_value = roberta_last_hidden_state_source

# Apply cross-attention
output, attention_weights = cross_attention_layer([input_query, input_key, input_value])

print("Input Query Shape:", input_query.shape)
print("Output Shape:", output.shape)
print("Attention Weights Shape:", attention_weights.shape)


Input Query Shape: (None, 128, 768)
Output Shape: (None, None, 64)
Attention Weights Shape: (None, 8, None, None)


In [87]:
# Extract the output from the RoBERTa model
#pooler_output = deberta_pooler_output
word_embeddings = output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 64]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

# Add BiGRU layers for word-level attention
gru_units = 64
dropout_rate = 0.2

output = Reshape((-1, gru_units))(word_attention_sum)
output = Bidirectional(GRU(gru_units, return_sequences=True))(output)
output = Dropout(dropout_rate)(output)
output = Bidirectional(GRU(gru_units, kernel_regularizer=regularizers.l2(0.01), return_sequences=True))(output)
output = Dropout(dropout_rate)(output)

# Apply sentence-level attention
sentence_attention_scores = Dense(1, activation='relu')(output)
sentence_attention_scores = Reshape((-1,))(sentence_attention_scores)
sentence_attention_weights = Activation('softmax')(sentence_attention_scores)
sentence_attention_weights = Reshape((-1, 1))(sentence_attention_weights)

# Tile the attention weights to match the dimensions of output
sentence_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 2 * gru_units]))(sentence_attention_weights)

# Apply element-wise multiplication between output and attention weights
sentence_attention = Multiply()([output, sentence_attention_weights])

# Sum up the sentence-level representations
sentence_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(sentence_attention)

# Apply global max pooling to capture the most important features
sentence_attention_pooling = GlobalMaxPooling1D()(sentence_attention)

combined_representation = tf.concat([sentence_attention_sum, sentence_attention_pooling], axis=1)
flat_labels=Flatten()(arr2_input)
dense_layer = tf.keras.layers.Dense(256, activation='relu')(flat_labels)
model1 = Model(inputs= [input_ids_source,token_type_ids_source,attention_mask_source,input_ids_reply,token_type_ids_reply,attention_mask_reply],outputs=[combined_representation])
dense_layer
dot_product=tf.keras.layers.Dot(axes=1)([model1.output, dense_layer])
#dot_product = tf.matmul(modelctR, dense_layer, transpose_b=True)

# Add dropout regularization after the hierarchical attention layer
#combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,dot_product], axis=1)
combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,dot_product], axis=1)

In [88]:
sentence_attention_sum.shape

TensorShape([None, 128])

In [89]:
sentence_attention_pooling.shape

TensorShape([None, 128])

In [90]:
euclidean_distances_train1_reshaped.shape

TensorShape([1, 8])

In [91]:
from tensorflow.keras.layers import Concatenate

# Reshape euclidean_distances_train to be a column vector
euclidean_distances_train = tf.expand_dims(euclidean_distances_train, axis=1)
euclidean_distances_train1 = euclidean_distances_train[:8]  # Take the first 8 elements

euclidean_distances_train1_reshaped = tf.reshape(euclidean_distances_train1, shape=(1, 8))

# Add Euclidean distance as a feature
combined_representation_with_distance = Concatenate(axis=1)([sentence_attention_sum, sentence_attention_pooling])#, euclidean_distances_train1_reshaped])

# Continue with the rest of your model
output = Dropout(dropout_rate)(combined_representation_with_distance)

In [ ]:
dot_product

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dot')>

In [ ]:
model1.output

<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'tf.concat')>

In [ ]:
output = Dropout(dropout_rate)(combined_representation)

# Add the final dense layer
output_units = 4
output1 = Dense(output_units, activation="softmax")(output)

# Create a new model with the modified architecture
#modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,arr2_input], outputs=output1)
modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,arr2_input], outputs=output1)

# Define the optimizer
learning_rate = 2e-5#1e-6
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)
#CategoricalCrossentropy
num_epochs = 60

In [ ]:
# Compile the model
modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [58]:
replicated_array_train = np.tile(arr2, (2081, 1,1))
replicated_array_val = np.tile(arr2, (232, 1,1))
replicated_array_val.shape

(232, 4, 768)

In [ ]:
#X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,clip_s_train,clip_r_train,replicated_array_train]
X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,replicated_array_train]
label_train = np.asarray(encoded_dataset_source['train']["label"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label"]).astype(np.float32)
# Train the modified model
#modified_model.fit(X,label_train,validation_data=([input_ids_valid_source,token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply,token_type_ids_valid_reply, attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,clip_s_val,clip_r_val,replicated_array_val],label_val), epochs=num_epochs,batch_size=8)
modified_model.fit(X,label_train,validation_data=([input_ids_valid_source,token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply,token_type_ids_valid_reply, attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,replicated_array_val],label_val), epochs=num_epochs,batch_size=8)

Epoch 1/60
261/261 [==============================] - 267s 616ms/step - loss: 4.1308 - accuracy: 0.4113 - val_loss: 3.8714 - val_accuracy: 0.4957
Epoch 2/60
261/261 [==============================] - 138s 527ms/step - loss: 3.7812 - accuracy: 0.5339 - val_loss: 3.5836 - val_accuracy: 0.6121
Epoch 3/60
261/261 [==============================] - 135s 516ms/step - loss: 3.4715 - accuracy: 0.6247 - val_loss: 3.3128 - val_accuracy: 0.6466
Epoch 4/60
261/261 [==============================] - 135s 516ms/step - loss: 3.1748 - accuracy: 0.6852 - val_loss: 3.0728 - val_accuracy: 0.6853
Epoch 5/60
261/261 [==============================] - 135s 517ms/step - loss: 2.9158 - accuracy: 0.7338 - val_loss: 2.8399 - val_accuracy: 0.7284
Epoch 6/60
261/261 [==============================] - 134s 513ms/step - loss: 2.6947 - accuracy: 0.7660 - val_loss: 2.6756 - val_accuracy: 0.7328
Epoch 7/60
261/261 [==============================] - 134s 513ms/step - loss: 2.4525 - accuracy: 0.8193 - val_loss: 2.5015 -

In [59]:
replicated_array_test = np.tile(arr2, (257, 1,1))

In [ ]:
# Evaluate the model on the test dataset
X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,v_p_s_test,v_neg_s_test,v_neu_s_test,v_p_r_test,v_neg_r_test,v_neu_r_test,replicated_array_test]
label_test = np.asarray(encoded_dataset_reply['test']["label"]).astype(np.float32)

test_results = modified_model.evaluate(X_test,label_test)

# Print the test accuracy
test_accuracy = test_results[1] * 100
print("Test Accuracy: {:.2f}%".format(test_accuracy))

9/9 [==============================] - 12s 458ms/step - loss: 1.0282 - accuracy: 0.8132
Test Accuracy: 81.32%


In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
# y_true = tf_test_dataset.map(lambda x, y: y)
# y_true = np.concatenate(list(y_true.as_numpy_iterator()))

# Calculate the F1 score
f1score = f1_score(label_test, y_pred, average='weighted')
f1_score_macro =precision_recall_fscore_support(label_test, y_pred, average='macro')

print("F1 Score: {:.2f}".format(f1score))
print("F1 macro Score: {:.2f}",(f1_score_macro))

9/9 [==============================] - 12s 461ms/step
F1 Score: 0.81
F1 macro Score: {:.2f} (0.8187274774774775, 0.8107516783887468, 0.8113546892419174, None)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
#y_true = tf_test_dataset.map(lambda x, y: y)
#y_true = np.concatenate(list(y_true.as_numpy_iterator()))
# Calculate the F1 score for each label
y_true = label_test
num_labels = 4 # Number of labels in the dataset
f1_scores = []
for label in range(num_labels):
    label_true = (y_true == label)
    label_pred = (y_pred == label)
    label_f1_score = f1_score(label_true, label_pred)
    f1_scores.append(label_f1_score)

# Print the F1 score for each label
for label, f1_score in enumerate(f1_scores):
    print("F1 Score for label {}: {:.2f}".format(label, f1_score))

9/9 [==============================] - 4s 472ms/step
F1 Score for label 0: 0.80
F1 Score for label 1: 0.81
F1 Score for label 2: 0.87
F1 Score for label 3: 0.77


In [ ]:
import tensorflow as tf

class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQ = tf.keras.layers.Dense(embed_size)
        self.WK = tf.keras.layers.Dense(embed_size)
        self.WV = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WO = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value = inputs

        batch_size = tf.shape(query)[0]

        # Linear projections
        query = self.WQ(query)
        key = self.WK(key)
        value = self.WV(value)

        # Split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention, attention_weights = self.scaled_dot_product_attention(query, key, value)

        # Merge heads
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embed_size))

        # Linear projection
        output = self.WO(concat_attention)

        return output, attention_weights

# Example usage
seq_len = 10
batch_size = 32
embed_size = 64
num_heads = 8

cross_attention_layer = CrossAttentionLayer(embed_size, num_heads)

# Dummy input tensors
input_query = roberta_last_hidden_state_source
input_key = roberta_last_hidden_state_reply
input_value = roberta_last_hidden_state_reply

# Apply cross-attention
output, attention_weights = cross_attention_layer([input_query, input_key, input_value])

print("Input Query Shape:", input_query.shape)
print("Output Shape:", output.shape)
print("Attention Weights Shape:", attention_weights.shape)


Input Query Shape: (None, 128, 768)
Output Shape: (None, None, 64)
Attention Weights Shape: (None, 8, None, None)


In [ ]:
# Extract the output from the RoBERTa model
#pooler_output = deberta_pooler_output
word_embeddings = output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 64]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

# Add BiGRU layers for word-level attention
gru_units = 64
dropout_rate = 0.2

output = Reshape((-1, gru_units))(word_attention_sum)
output = Bidirectional(GRU(gru_units, return_sequences=True))(output)
output = Dropout(dropout_rate)(output)
output = Bidirectional(GRU(gru_units, kernel_regularizer=regularizers.l2(0.01), return_sequences=True))(output)
output = Dropout(dropout_rate)(output)

# Apply sentence-level attention
sentence_attention_scores = Dense(1, activation='relu')(output)
sentence_attention_scores = Reshape((-1,))(sentence_attention_scores)
sentence_attention_weights = Activation('softmax')(sentence_attention_scores)
sentence_attention_weights = Reshape((-1, 1))(sentence_attention_weights)

# Tile the attention weights to match the dimensions of output
sentence_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 2 * gru_units]))(sentence_attention_weights)

# Apply element-wise multiplication between output and attention weights
sentence_attention = Multiply()([output, sentence_attention_weights])

# Sum up the sentence-level representations
sentence_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(sentence_attention)

# Apply global max pooling to capture the most important features
sentence_attention_pooling = GlobalMaxPooling1D()(sentence_attention)

combined_representation = tf.concat([sentence_attention_sum, sentence_attention_pooling], axis=1)
flat_labels=Flatten()(arr2_input)
dense_layer = tf.keras.layers.Dense(256, activation='relu')(flat_labels)
model1 = Model(inputs= [input_ids_source,token_type_ids_source,attention_mask_source,input_ids_reply,token_type_ids_reply,attention_mask_reply],outputs=[combined_representation])
dense_layer
dot_product=tf.keras.layers.Dot(axes=1)([model1.output, dense_layer])
#dot_product = tf.matmul(modelctR, dense_layer, transpose_b=True)

# Add dropout regularization after the hierarchical attention layer
#combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,dot_product], axis=1)
combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,dot_product], axis=1)

In [ ]:
output = Dropout(dropout_rate)(combined_representation)

# Add the final dense layer
output_units = 4
output1 = Dense(output_units, activation="softmax")(output)

# Create a new model with the modified architecture
#modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,arr2_input], outputs=output1)
modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,arr2_input], outputs=output1)

# Define the optimizer
learning_rate = 2e-5#1e-6
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)
#CategoricalCrossentropy
num_epochs = 60

In [ ]:
# Compile the model
modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
#X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,clip_s_train,clip_r_train,replicated_array_train]
X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,replicated_array_train]
label_train = np.asarray(encoded_dataset_source['train']["label"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label"]).astype(np.float32)
# Train the modified model
#modified_model.fit(X,label_train,validation_data=([input_ids_valid_source,token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply,token_type_ids_valid_reply, attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,clip_s_val,clip_r_val,replicated_array_val],label_val), epochs=num_epochs,batch_size=8)
modified_model.fit(X,label_train,validation_data=([input_ids_valid_source,token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply,token_type_ids_valid_reply, attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,replicated_array_val],label_val), epochs=num_epochs,batch_size=8)

Epoch 1/60


KeyboardInterrupt: ignored

In [50]:
# Extract the output from the RoBERTa model
#pooler_output = deberta_pooler_output
word_embeddings = output

# Apply word-level attention
word_attention_scores = Dense(1, activation='relu')(word_embeddings)
word_attention_scores
word_attention_scores = Reshape((-1,))(word_attention_scores)
word_attention_weights = Activation('softmax')(word_attention_scores)
word_attention_weights = Reshape((-1, 1))(word_attention_weights)

# Tile the attention weights to match the dimensions of word_embeddings
word_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 64]))(word_attention_weights)

# Apply element-wise multiplication between word embeddings and attention weights
word_attention = Multiply()([word_embeddings, word_attention_weights])

# Sum up the word-level representations
word_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(word_attention)

# Add BiGRU layers for word-level attention
gru_units = 64
dropout_rate = 0.2

output = Reshape((-1, gru_units))(word_attention_sum)
output = Bidirectional(GRU(gru_units, return_sequences=True))(output)
output = Dropout(dropout_rate)(output)
output = Bidirectional(GRU(gru_units, kernel_regularizer=regularizers.l2(0.01), return_sequences=True))(output)
output = Dropout(dropout_rate)(output)

# Apply sentence-level attention
sentence_attention_scores = Dense(1, activation='relu')(output)
sentence_attention_scores = Reshape((-1,))(sentence_attention_scores)
sentence_attention_weights = Activation('softmax')(sentence_attention_scores)
sentence_attention_weights = Reshape((-1, 1))(sentence_attention_weights)

# Tile the attention weights to match the dimensions of output
sentence_attention_weights = Lambda(lambda x: tf.tile(x, [1, 1, 2 * gru_units]))(sentence_attention_weights)

# Apply element-wise multiplication between output and attention weights
sentence_attention = Multiply()([output, sentence_attention_weights])

# Sum up the sentence-level representations
sentence_attention_sum = Lambda(lambda x: tf.reduce_sum(x, axis=1))(sentence_attention)

# Apply global max pooling to capture the most important features
sentence_attention_pooling = GlobalMaxPooling1D()(sentence_attention)

combined_representation = tf.concat([sentence_attention_sum, sentence_attention_pooling], axis=1)
flat_labels=Flatten()(arr2_input)
dense_layer = tf.keras.layers.Dense(256, activation='relu')(flat_labels)
model1 = Model(inputs= [input_ids_source,token_type_ids_source,attention_mask_source,input_ids_reply,token_type_ids_reply,attention_mask_reply],outputs=[combined_representation])
dense_layer
dot_product=tf.keras.layers.Dot(axes=1)([model1.output, dense_layer])
#dot_product = tf.matmul(modelctR, dense_layer, transpose_b=True)

# Add dropout regularization after the hierarchical attention layer
#combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,dot_product], axis=1)
combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,dot_product], axis=1)

ValueError: ignored

In [ ]:
# Add Dense layer to reduce the dimensionality of dense_layer
dense_layer_reduced = Dense(128, activation='relu')(dense_layer)

# Calculate Euclidean distance between sentence_attention_sum and dense_layer_reduced
euclidean_distance = Lambda(lambda x: tf.norm(x[0] - x[1], axis=1, keepdims=True))([sentence_attention_sum, dense_layer_reduced])

# Add Euclidean distance as a feature
combined_representation_with_distance = tf.concat([sentence_attention_sum, sentence_attention_pooling, euclidean_distance], axis=1)

# Continue with the rest of your model
output = Dropout(dropout_rate)(combined_representation_with_distance)

In [92]:
# Add the final dense layer
output_units = 4
output1 = Dense(output_units, activation="softmax")(output)

# Create a new model with the modified architecture
#modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,arr2_input], outputs=output1)
modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,arr2_input,euclidean_distances_input], outputs=output1)

# Define the optimizer
learning_rate = 2e-5#1e-6
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)
#CategoricalCrossentropy
num_epochs = 35

In [93]:
# Compile the model
modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [96]:
# Assuming your PyTorch tensor is euclidean_distances_valid_torch
euclidean_distances_valid_np = euclidean_distances_valid.numpy()
euclidean_distances_valid_tf = tf.convert_to_tensor(euclidean_distances_valid_np, dtype=tf.float32)

euclidean_distances_train_np = euclidean_distances_train.numpy()
euclidean_distances_train_tf = tf.convert_to_tensor(euclidean_distances_train_np, dtype=tf.float32)

In [98]:
#X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,clip_s_train,clip_r_train,replicated_array_train]
X = [input_ids_train_source,token_type_ids_train_source, attention_mask_train_source,input_ids_train_reply,token_type_ids_train_reply, attention_mask_train_reply,v_p_s_train,v_neg_s_train,v_neu_s_train,v_p_r_train,v_neg_r_train,v_neu_r_train,replicated_array_train,euclidean_distances_train_tf]
label_train = np.asarray(encoded_dataset_source['train']["label"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label"]).astype(np.float32)
# Train the modified model
#modified_model.fit(X,label_train,validation_data=([input_ids_valid_source,token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply,token_type_ids_valid_reply, attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,clip_s_val,clip_r_val,replicated_array_val],label_val), epochs=num_epochs,batch_size=8)
modified_model.fit(X,label_train,validation_data=([input_ids_valid_source,token_type_ids_valid_source, attention_mask_valid_source,input_ids_valid_reply,token_type_ids_valid_reply, attention_mask_valid_reply,v_p_s_v, v_neg_s_v, v_neu_s_v ,v_p_r_v, v_neg_r_v, v_neu_r_v,replicated_array_val,euclidean_distances_valid_tf],label_val), epochs=num_epochs,batch_size=8)

Epoch 1/35
261/261 [==============================] - 143s 547ms/step - loss: 3.7991 - accuracy: 0.5579 - val_loss: 3.6229 - val_accuracy: 0.5948
Epoch 2/35
261/261 [==============================] - 133s 510ms/step - loss: 3.5518 - accuracy: 0.5954 - val_loss: 3.3955 - val_accuracy: 0.6121
Epoch 3/35
261/261 [==============================] - 135s 516ms/step - loss: 3.3406 - accuracy: 0.6055 - val_loss: 3.1899 - val_accuracy: 0.6250
Epoch 4/35
261/261 [==============================] - 134s 513ms/step - loss: 3.1472 - accuracy: 0.6175 - val_loss: 3.0325 - val_accuracy: 0.6336
Epoch 5/35
261/261 [==============================] - 133s 511ms/step - loss: 2.9754 - accuracy: 0.6237 - val_loss: 2.8903 - val_accuracy: 0.6336
Epoch 6/35
261/261 [==============================] - 132s 507ms/step - loss: 2.8076 - accuracy: 0.6506 - val_loss: 2.7244 - val_accuracy: 0.6595
Epoch 7/35
261/261 [==============================] - 133s 509ms/step - loss: 2.6010 - accuracy: 0.6977 - val_loss: 2.5477 -

In [99]:
euclidean_distances_test_np = euclidean_distances_test.numpy()
euclidean_distances_test_tf = tf.convert_to_tensor(euclidean_distances_test_np, dtype=tf.float32)

In [100]:
# Evaluate the model on the test dataset
X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,v_p_s_test,v_neg_s_test,v_neu_s_test,v_p_r_test,v_neg_r_test,v_neu_r_test,replicated_array_test,euclidean_distances_test_tf]
label_test = np.asarray(encoded_dataset_source['test']["label"]).astype(np.float32)

test_results = modified_model.evaluate(X_test,label_test)

# Print the test accuracy
test_accuracy = test_results[1] * 100
print("Test Accuracy: {:.2f}%".format(test_accuracy))

9/9 [==============================] - 14s 453ms/step - loss: 0.9582 - accuracy: 0.8599
Test Accuracy: 85.99%


In [101]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
# y_true = tf_test_dataset.map(lambda x, y: y)
# y_true = np.concatenate(list(y_true.as_numpy_iterator()))

# Calculate the F1 score
f1score = f1_score(label_test, y_pred, average='weighted')
f1_score_macro =precision_recall_fscore_support(label_test, y_pred, average='macro')

print("F1 Score: {:.2f}".format(f1score))
print("F1 macro Score: {:.2f}",(f1_score_macro))

9/9 [==============================] - 12s 455ms/step
F1 Score: 0.86
F1 macro Score: {:.2f} (0.86086204537486, 0.8592146777189137, 0.8587452599080506, None)


In [102]:
import numpy as np
from sklearn.metrics import f1_score

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
#y_true = tf_test_dataset.map(lambda x, y: y)
#y_true = np.concatenate(list(y_true.as_numpy_iterator()))
# Calculate the F1 score for each label
y_true = label_test
num_labels = 4 # Number of labels in the dataset
f1_scores = []
for label in range(num_labels):
    label_true = (y_true == label)
    label_pred = (y_pred == label)
    label_f1_score = f1_score(label_true, label_pred)
    f1_scores.append(label_f1_score)

# Print the F1 score for each label
for label, f1_score in enumerate(f1_scores):
    print("F1 Score for label {}: {:.2f}".format(label, f1_score))

9/9 [==============================] - 4s 458ms/step
F1 Score for label 0: 0.84
F1 Score for label 1: 0.85
F1 Score for label 2: 0.91
F1 Score for label 3: 0.82


In [ ]:
'''# Calculate Euclidean distance between sentence_attention_sum and dense_layer
euclidean_distance = Lambda(lambda x: tf.norm(x[0] - x[1], axis=1, keepdims=True))([sentence_attention_sum, dense_layer])

# Add Euclidean distance as a feature
combined_representation_with_distance = tf.concat([sentence_attention_sum, sentence_attention_pooling, euclidean_distance], axis=1)

# Continue with the rest of your model
output = Dropout(dropout_rate)(combined_representation_with_distance)

In [ ]:
from IPython.display import clear_output
!pip install -q tensorflow==2.3.1
clear_output()

In [ ]:
# install tensorflow quantum
!pip install -q tensorflow_quantum
clear_output()


In [ ]:
!pip install tensorflow-gpu==2.1.0
!pip install cirq==0.7.0 pathos==0.2.5 tensorflow-quantum==0.2.0

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.1.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 0.3.1.27 Requires-Python ==2.7.*; 0.5.555 Requires-Python >=2.7,<=3.5; 0.5.556 Requires-Python ==3.5.*
ERROR: Could not find a version that satisfies the requirement tensorflow-quantum==0.2.0 (from versions: none)
ERROR: No matching distribution found for tensorflow-quantum==0.2.0


In [ ]:
import tensorflow_quantum as tfq

ModuleNotFoundError: ignored

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq
import numpy as np
from tensorflow.keras.layers import Dense, Reshape, Activation, Lambda, Multiply, Bidirectional, GRU, Dropout, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Define a simple quantum circuit layer
def create_quantum_circuit(qubits):
    return tfq.layers.PQC(tfq.util.random_pauli_circuit(qubits), [tf.keras.layers.Dense(1)])

# Quantum CNN layer
def quantum_cnn_layer(inputs):
    qubits = 4  # Adjust the number of qubits as needed
    quantum_input = tfq.convert_to_tensor([inputs])  # Convert to quantum tensor
    quantum_circuit = create_quantum_circuit(qubits)(quantum_input)
    return tfq.layers.Expectation()(quantum_circuit)



# Example usage
quantum_cnn_output = quantum_cnn_layer(output)

# Continue with the rest of your model
output = Dropout(dropout_rate)(combined_representation)

# Add the final dense layer
output_units = 4
output1 = Dense(output_units, activation="softmax")(output)

# Create a new model with the modified architecture
modified_model = Model(inputs=[model1.input,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,arr2_input], outputs=output1)

# Define the optimizer
learning_rate = 2e-5
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)

# Compile the model
modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# Assuming you have prepared your quantum-ready dataset
# X_quantum should contain quantum-ready data for the quantum layer
X_quantum = np.random.rand(100, 10)  # Replace with your actual quantum-ready data

# Train the modified model
label_train = np.asarray(encoded_dataset_source['train']["label"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["label"]).astype(np.float32)
modified_model.fit([X, X_quantum, v_p_s_train, v_neg_s_train, v_neu_s_train, v_p_r_train, v_neg_r_train, v_neu_r_train, replicated_array_train],
                   label_train,
                   validation_data=([input_ids_valid_source, token_type_ids_valid_source, attention_mask_valid_source,
                                     input_ids_valid_reply, token_type_ids_valid_reply, attention_mask_valid_reply,
                                     v_p_s_v, v_neg_s_v, v_neu_s_v, v_p_r_v, v_neg_r_v, v_neu_r_v, replicated_array_val],
                                    label_val),
                   epochs=num_epochs, batch_size=8)


ModuleNotFoundError: ignored